In [3]:
%pip install scikit-learn

    pandas (>=1.0.*)
            ~~~~~~^
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

df = pd.read_csv('/projects/diffusion_bootcamp/datasets/ibmaml/HI-Small_Trans.csv')
df.head()


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


# Explore Data

In [2]:
df.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering'],
      dtype='object')

In [7]:
print(len(set(df['Account'])))

print(df.shape)


print(len(set(df['Account.1'])))



496995
(5078345, 11)
420636


In [22]:
# Feature Engineering 1: identical account activity
df["Identitcal Account Activity"] = ((df["Account"] == df["Account.1"]) & (df["From Bank"] == df["To Bank"])).astype(int)


In [49]:
# ((df["Account"] == df["Account.1"]) & (df["From Bank"] == df["To Bank"]) & (df["Payment Format"] != "Reinvestment"))

filtered_df = df[
    (df['Account'] == df['Account.1']) &
    (df['From Bank'] == df['To Bank']) &
    (df['Payment Format'] != 'Reinvestment')
]

filtered_df

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Same Account Activity,Same Bank Activity,Identitcal Account Activity,Datetime,SecondsSinceEpoch
105,2022/09/01 00:26,12,800043C00,12,800043C00,755.250000,US Dollar,755.250000,US Dollar,Cheque,0,1,1,1,2022-09-01 00:26:00,1661991960
106,2022/09/01 00:20,12,800043C00,12,800043C00,876.160000,US Dollar,876.160000,US Dollar,Credit Card,0,1,1,1,2022-09-01 00:20:00,1661991600
109,2022/09/01 00:20,10,800056660,10,800056660,4461.810000,US Dollar,4461.810000,US Dollar,Cheque,0,1,1,1,2022-09-01 00:20:00,1661991600
110,2022/09/01 00:03,10,800056660,10,800056660,1134.320000,US Dollar,1134.320000,US Dollar,Credit Card,0,1,1,1,2022-09-01 00:03:00,1661990580
111,2022/09/01 00:18,10,800056660,10,800056660,726.940000,US Dollar,726.940000,US Dollar,Cash,0,1,1,1,2022-09-01 00:18:00,1661991480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5078269,2022/09/10 23:35,126,813B6E021,126,813B6E021,0.213793,Bitcoin,0.213793,Bitcoin,Bitcoin,0,1,1,1,2022-09-10 23:35:00,1662852900
5078270,2022/09/10 23:59,126,813B6E021,126,813B6E021,0.121724,Bitcoin,0.121724,Bitcoin,Bitcoin,0,1,1,1,2022-09-10 23:59:00,1662854340
5078271,2022/09/10 23:38,15,813B765F1,15,813B765F1,0.003384,Bitcoin,0.003384,Bitcoin,Bitcoin,0,1,1,1,2022-09-10 23:38:00,1662853080
5078272,2022/09/10 23:37,15,813B765F1,15,813B765F1,0.006543,Bitcoin,0.006543,Bitcoin,Bitcoin,0,1,1,1,2022-09-10 23:37:00,1662853020


In [42]:
# Feature Engineering 2: Timestamp normalization
df['Datetime'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M')
df["SecondsSinceEpoch"] = (df['Datetime'].astype(int) // 10**9)


In [50]:
# Final DF
feature_engineered_df = df[["SecondsSinceEpoch", "From Bank", "Account","To Bank","Account.1", "Identitcal Account Activity", "Amount Received", "Receiving Currency", "Amount Paid", "Payment Currency", "Payment Format", "Is Laundering"]]
feature_engineered_df

,SecondsSinceEpoch,From Bank,Account,To Bank,Account.1,Identitcal Account Activity,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,1661991600,10,8000EBD30,10,8000EBD30,1,3697.340000,US Dollar,3697.340000,US Dollar,Reinvestment,0
1,1661991600,3208,8000F4580,1,8000F5340,0,0.010000,US Dollar,0.010000,US Dollar,Cheque,0
2,1661990400,3209,8000F4670,3209,8000F4670,1,14675.570000,US Dollar,14675.570000,US Dollar,Reinvestment,0
3,1661990520,12,8000F5030,12,8000F5030,1,2806.970000,US Dollar,2806.970000,US Dollar,Reinvestment,0
4,1661990760,10,8000F5200,10,8000F5200,1,36682.970000,US Dollar,36682.970000,US Dollar,Reinvestment,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5078340,1662854220,54219,8148A6631,256398,8148A8711,0,0.154978,Bitcoin,0.154978,Bitcoin,Bitcoin,0
5078341,1662852900,15,8148A8671,256398,8148A8711,0,0.108128,Bitcoin,0.108128,Bitcoin,Bitcoin,0
5078342,1662853920,154365,8148A6771,256398,8148A8711,0,0.004988,Bitcoin,0.004988,Bitcoin,Bitcoin,0
5078343,1662853560,256398,8148A6311,256398,8148A8711,0,0.038417,Bitcoin,0.038417,Bitcoin,Bitcoin,0


In [46]:
# Group by 'From Bank' and 'To Bank' and sum the 'Is Laundering' column
result = df.groupby(['From Bank', 'To Bank'])['Is Laundering'].sum().reset_index()

# Rename the column for clarity
result = result.rename(columns={'Is Laundering': 'Total Laundering Count'})

# Sort the results by 'Total Laundering Count' in descending order
result = result.sort_values(by='Total Laundering Count', ascending=False)

result

df['From Bank'].nunique()
df['To Bank'].nunique()

,From Bank,To Bank,Total Laundering Count
72982,15231,23691,27
103670,23691,15231,20
161188,49365,119,18
187865,148350,148350,16
7923,119,49365,15
...,...,...,...
96132,21611,231706,0
96133,21611,231786,0
96134,21611,232555,0
96135,21611,232583,0


# Data sampling for Fanout

In [5]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class BankAccountMerger(BaseEstimator, TransformerMixin):
    def __init__(self, source_bank_col, source_account_col, dest_bank_col, dest_account_col):
        self.source_bank_col = source_bank_col
        self.source_account_col = source_account_col
        self.dest_bank_col = dest_bank_col
        self.dest_account_col = dest_account_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        # Convert to string and then concatenate
        X['Source'] = X[self.source_bank_col].astype(str) + '_' + X[self.source_account_col].astype(str)
        X['Destination'] = X[self.dest_bank_col].astype(str) + '_' + X[self.dest_account_col].astype(str)
        return X.drop([self.source_bank_col, self.source_account_col, 
                       self.dest_bank_col, self.dest_account_col], axis=1)

class SourceAccountSampler(BaseEstimator, TransformerMixin):
    def __init__(self, source_col, sample_size=100, random_state=None):
        self.source_col = source_col
        self.sample_size = sample_size
        self.random_state = random_state

    def fit(self, X, y=None):
        unique_sources = X[self.source_col].unique()
        np.random.seed(self.random_state)
        self.sampled_sources = np.random.choice(unique_sources, 
                                                size=min(self.sample_size, len(unique_sources)), 
                                                replace=False)
        return self

    def transform(self, X):
        return X[X[self.source_col].isin(self.sampled_sources)]

# Create the pipeline
pipeline = Pipeline([
    ('merge_accounts', BankAccountMerger('From Bank', 'Account', 'To Bank', 'Account.1')),
    ('sample_sources', SourceAccountSampler('Source', sample_size=100, random_state=42))
])

# Assuming your DataFrame is called 'df'
# Apply the pipeline
result_df = pipeline.fit_transform(df)

In [7]:
result_df['Receiving Currency'].unique()

array(['US Dollar', 'Euro', 'Yuan', 'Rupee', 'Ruble', 'Canadian Dollar',
       'Australian Dollar', 'Brazil Real', 'Swiss Franc', 'Shekel',
       'Bitcoin', 'Yen', 'UK Pound'], dtype=object)

In [6]:
result_df.result_df

,Timestamp,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Source,Destination
8027,2022/09/01 00:27,1111.57,US Dollar,1111.57,US Dollar,Reinvestment,0,31125_800DBF480,31125_800DBF480
18113,2022/09/01 00:16,470355.71,US Dollar,470355.71,US Dollar,Reinvestment,0,10_802144AF0,10_802144AF0
18595,2022/09/01 00:22,17.30,US Dollar,17.30,US Dollar,Reinvestment,0,23402_802273B70,23402_802273B70
29192,2022/09/01 00:21,16146.25,US Dollar,16146.25,US Dollar,Reinvestment,0,4503_8039C1A90,4503_8039C1A90
42649,2022/09/01 00:26,9876.76,US Dollar,9876.76,US Dollar,Reinvestment,0,315574_80603B220,315574_80603B220
